In [7]:
import pandas as pd
from sklearn.cluster import KMeans
import math
import matplotlib.pyplot as plt
import warnings
import numpy as np
import sqlite3 as sql
from mlxtend.frequent_patterns import apriori, association_rules
warnings.simplefilter('ignore')

ModuleNotFoundError: No module named 'mlxtend'

In [2]:
conn = sql.connect('..\Databases\GreatOutdoors\go_sales.sqlite')
query = "SELECT * FROM product"
query2 = "SELECT * FROM order_details"
df = pd.read_sql(query,conn)
df2 = pd.read_sql(query2,conn)

df = df.merge(df2, on='PRODUCT_NUMBER')

df

,PRODUCT_NUMBER,INTRODUCTION_DATE,PRODUCT_TYPE_CODE,PRODUCTION_COST,MARGIN,PRODUCT_IMAGE,LANGUAGE,PRODUCT_NAME,DESCRIPTION,TRIAL888,ORDER_DETAIL_CODE,ORDER_NUMBER,QUANTITY,UNIT_COST,UNIT_PRICE,UNIT_SALE_PRICE,TRIAL879
0,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100085,8474,40,4.01,6.03,6.03,T
1,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100086,8470,40,4.01,6.03,6.03,T
2,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100087,8464,48,4.01,6.03,6.03,T
3,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100088,9254,66,4.01,6.03,6.03,T
4,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100089,8472,88,4.01,6.03,6.03,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43058,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",T,98478,8210,30,1.97,4.94,4.94,T
43059,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",T,99566,8246,20,2,5,3.95,T
43060,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",T,99567,8239,18,2,5,5,T
43061,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",T,99568,8270,26,2,5,5,T


In [5]:
df = df.loc[:,['PRODUCT_NUMBER','PRODUCT_TYPE_CODE','UNIT_PRICE']]
df['PRODUCT_NUMBER'] = pd.to_numeric(df['PRODUCT_NUMBER'], errors='coerce')
df['PRODUCT_TYPE_CODE'] = pd.to_numeric(df['PRODUCT_TYPE_CODE'], errors='coerce')
df['UNIT_PRICE'] = pd.to_numeric(df['UNIT_PRICE'], errors='coerce')
df

,PRODUCT_NUMBER,PRODUCT_TYPE_CODE,UNIT_PRICE
0,1,1,6.03
1,1,1,6.03
2,1,1,6.03
3,1,1,6.03
4,1,1,6.03
...,...,...,...
43058,99,17,4.94
43059,99,17,5.00
43060,99,17,5.00
43061,99,17,5.00
